In [1]:
import numpy as np
import math
import os
import sys
import progressbar

dimension = 39


In [2]:
def get_confusion_mat(true_class, pred_class):
    conf_mat = np.full((3,3),1e-10)
    for i in range(len(true_class)):
        conf_mat[true_class[i]][pred_class[i]]+=1
    return np.array(conf_mat)

In [3]:
def get_metric(conf_matrix):
    x = conf_matrix.shape[0]
    precision = []
    accuracy = []
    recall = []
    f_measure = []
    total_sum = np.sum(conf_matrix)
    for i in range(x):
        TP = conf_matrix[i][i]
        FN = np.sum(conf_matrix[i,:])-conf_matrix[i][i]
        FP = np.sum(conf_matrix[:,i])-conf_matrix[i][i]
        TN = total_sum-(TP+FP+FN)

        recal = TP/(TP+FN)
        preci = TP/(TP+FP)
        acc = (TP+TN)/(TP+TN+FP+FN)
        
        precision.append(preci)
        recall.append(recal)
        accuracy.append(acc)
        f_measure.append((2*recal*preci)/(recal+preci))
        
    return accuracy,precision,recall,f_measure

In [4]:
def find_distance(data1, data2):
    su = 0.0
    for i in range(dimension):
        su += ((data1[i]-data2[i])**2);
    return su
        
    

In [5]:
def dtw(data1, data2):
    
    n1 = data1.shape[0]
    n2 = data2.shape[0]
    
    dp = np.ndarray(shape=(n1,n2),dtype=float)
    for i in range(n1):
        dp[i][0] = find_distance(data1[i,:], data2[0,:])
    for i in range(n2):
        dp[0][i] = find_distance(data1[0,:], data2[i,:])
        
    for i in range(1,n1):
        for j in range(1,n2):
            dp[i][j] = find_distance(data1[i,:],data2[j,:]) + min(dp[i-1][j],dp[i-1][j-1],dp[i][j-1])
    
    return dp[n1-1][n2-1]
            

In [6]:
def get_predictions(path, st,k):
    pred_class = []
    for f in progressbar.progressbar(os.listdir(os.path.join(path,st))):
        data_test = np.load(os.path.join(os.path.join(path,st),f))
        lis = []
        for f_train in os.listdir(os.path.join('Group13/Train','di_npy')):
            data_train = np.load(os.path.join(os.path.join('Group13/Train','di_npy'),f_train))
            dist = dtw(data_train, data_test)
            lis.append([dist,0])
        for f_train in os.listdir(os.path.join('Group13/Train','dI_npy')):
            data_train = np.load(os.path.join(os.path.join('Group13/Train','dI_npy'),f_train))
            dist = dtw(data_train, data_test)
            lis.append([dist,1])
        for f_train in os.listdir(os.path.join('Group13/Train','DI_npy')):
            data_train = np.load(os.path.join(os.path.join('Group13/Train','DI_npy'),f_train))
            dist = dtw(data_train, data_test)
            lis.append([dist,2])
        lis2 = sorted(lis,key=lambda x: x[0])
        coun = [0,0,0]
        for i in range(k):
            coun[lis2[i][1]]+=1
        pred_class.append(np.argmax(coun))
        
    return pred_class
    

In [13]:
def main():
    K = [4, 8, 16, 32]
    path = 'Group13/Test'
    true_class = []
    for f in os.listdir(os.path.join(path,'di_npy')):
        true_class.append(0)
    for f in os.listdir(os.path.join(path,'dI_npy')):
        true_class.append(1)
    for f in os.listdir(os.path.join(path,'DI_npy')):
        true_class.append(2)
        
    for k in K:
        pred_class = []
        p_class1 = get_predictions(path,'di_npy',k)
        p_class2 = get_predictions(path,'dI_npy',k)
        p_class3 = get_predictions(path,'DI_npy',k)
        for i in range(len(p_class1)):
            pred_class.append(p_class1[i])
        for i in range(len(p_class2)):
            pred_class.append(p_class2[i])
        for i in range(len(p_class3)):
            pred_class.append(p_class3[i])
        
        
        conf_mat = get_confusion_mat(true_class, pred_class)
        accuracy,precision,recall,f_measure = get_metric(conf_mat)
        print('#'*10,"Done for K: {}".format(k),'#'*10)
        print(conf_mat)
        print("Accuracy:\t ", round(accuracy[0],4), round(accuracy[1],4), round(accuracy[2],4))
        print("Precision:\t ",round(precision[0],4), round(precision[1],4), round(precision[2],4))
        print("Recall:\t ",   round(recall[0],4), round(recall[1],4), round(recall[2],2))
        print("F_measure:\t ",round(f_measure[0],4), round(f_measure[1],4), round(f_measure[2],4))
        
        

In [14]:
if(__name__=="__main__"):
    main()

100% (27 of 27) |########################| Elapsed Time: 0:01:25 Time:  0:01:25
100% (24 of 24) |########################| Elapsed Time: 0:01:26 Time:  0:01:26
100% (14 of 14) |########################| Elapsed Time: 0:00:42 Time:  0:00:42
N/A% (0 of 27) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--

########## Done for K: 4 ##########
[[ 23.   1.   3.]
 [ 15.   2.   7.]
 [  3.   1.  10.]]
Accuracy:	  0.6615 0.6308 0.7846
Precision:	  0.561 0.5 0.5
Recall:	  0.8519 0.0833 0.71
F_measure:	  0.6765 0.1429 0.5882


100% (27 of 27) |########################| Elapsed Time: 0:01:17 Time:  0:01:17
100% (24 of 24) |########################| Elapsed Time: 0:01:31 Time:  0:01:31
100% (14 of 14) |########################| Elapsed Time: 0:00:42 Time:  0:00:42
N/A% (0 of 27) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--

########## Done for K: 8 ##########
[[  2.30000000e+01   1.00000000e-10   4.00000000e+00]
 [  1.70000000e+01   1.00000000e+00   6.00000000e+00]
 [  6.00000000e+00   3.00000000e+00   5.00000000e+00]]
Accuracy:	  0.5846 0.6 0.7077
Precision:	  0.5 0.25 0.3333
Recall:	  0.8519 0.0417 0.36
F_measure:	  0.6301 0.0714 0.3448


100% (27 of 27) |########################| Elapsed Time: 0:01:17 Time:  0:01:17
100% (24 of 24) |########################| Elapsed Time: 0:01:28 Time:  0:01:28
100% (14 of 14) |########################| Elapsed Time: 0:00:42 Time:  0:00:42
N/A% (0 of 27) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--

########## Done for K: 16 ##########
[[  2.50000000e+01   1.00000000e-10   2.00000000e+00]
 [  1.90000000e+01   1.00000000e-10   5.00000000e+00]
 [  7.00000000e+00   3.00000000e+00   4.00000000e+00]]
Accuracy:	  0.5692 0.5846 0.7385
Precision:	  0.4902 0.0 0.3636
Recall:	  0.9259 0.0 0.29
F_measure:	  0.641 0.0 0.32


100% (27 of 27) |########################| Elapsed Time: 0:01:23 Time:  0:01:23
100% (24 of 24) |########################| Elapsed Time: 0:01:27 Time:  0:01:27
100% (14 of 14) |########################| Elapsed Time: 0:00:42 Time:  0:00:42


########## Done for K: 32 ##########
[[  2.70000000e+01   1.00000000e-10   1.00000000e-10]
 [  2.10000000e+01   1.00000000e-10   3.00000000e+00]
 [  1.10000000e+01   1.00000000e-10   3.00000000e+00]]
Accuracy:	  0.5077 0.6308 0.7846
Precision:	  0.4576 0.3333 0.5
Recall:	  1.0 0.0 0.21
F_measure:	  0.6279 0.0 0.3
